In [157]:
import tushare as ts 
import pandas as pd
from datetime import datetime,timedelta

class Stock_Select():
    def __init__(self,index_code,total_mv,share_ratio):
        self.pro = ts.pro_api()
        self.stock_list = []
        self.total_mv = total_mv
        self.share_ratio = share_ratio 
        self.index_code = index_code
        self.fina = pd.DataFrame()
        self.get_sw_codes()
    def get_sw_codes(self):
        df_stock= pro.index_member(index_code=self.index_code)
        self.stock_list = df_stock.con_code.to_list()
        return self.stock_list
    def get_code(self):
        date = (datetime.now() + timedelta(days=-1)).strftime('%Y%m%d') #当前日期的前一天，转为字符串 
        data = pd.DataFrame()   
        for i in self.stock_list:
            df_db= pro.query('daily_basic', ts_code=i, trade_date=date,fields='ts_code,trade_date,\
            turnover_rate,volume_ratio,pe,pb,total_share,free_share,total_mv')
            data =data.append(df_db)
        df_db =   data.copy() 
        df_db['share_ratio'] = df_db.free_share / df_db.total_share * 100 #计算自由流通股占比
        df_db['total_mv'] = df_db.total_mv  / 10000 #把总市值单位转为亿元
        filt = (df_db['total_mv'] > 50 ) & (df_db['share_ratio'] > 20)  #总市值 > 100亿,流通股占比大于20%
        df_db =  df_db[filt] 
        c_1 = set(df_db.ts_code.to_list()) #取set
        
        df = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
        year = (datetime.now() +timedelta(days=-365*3)).strftime('%Y%m%d') #上市时间为三年前的今天
        #保留上市时间大于三年个股数据
        df=df[df.list_date<year]
        #排除银行、保险、多元金融公司
        df=df[-df.industry.isin(['银行','保险','多元金融'])]
        #排除st和*ST股
        df=df[-df.name.str.startswith(('ST'))]
        df=df[-df.name.str.startswith(('*'))] 
        c_2 = set(df.ts_code.to_list())

        c = c_1 & c_2 #求交集
        df = df[df.ts_code.isin(c)]
        code=df.ts_code.values
        name=df.name.values
        return dict(zip(name,code))
    def get_indicators(self,code):
        #获取当前时间，计算当前和过去四年时间
        t0=datetime.now()
        t1=datetime(t0.year-4,t0.month,t0.day)
        end=t0.strftime('%Y%m%d')
        start=t1.strftime('%Y%m%d')
        
         #财务比率
        fields='ann_date,end_date,tr_yoy,op_yoy,\
             grossprofit_margin,expense_of_sales,inv_turn,eps,\
             ocfps,roe_yearly,roa2_yearly,netprofit_yoy'
        fina = (self.pro.fina_indicator(ts_code=code,start_date=start, end_date=end,fields=fields)
               .drop_duplicates(subset=['ann_date','end_date'],keep='first'))
        fina.set_index('end_date',inplace = True)
        fina=fina.sort_index()
        fina['eps_yearly'] = fina['eps'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        fina['inv_turn_yearly'] = fina['inv_turn'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        fina['ocfps_yearly'] = fina['ocfps'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        fina['expense_of_sales_yearly'] = fina['expense_of_sales'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]
        '''
         {'年化每股收益':'eps_yearly','年化存货周转率':'inv_turn_yearly','年化每股经营性现金流量':'ocfps_yearly',\
         '毛利率':'grossprofit_margin','年化期间费用率':'expense_of_sales_yearly',\
          '年化净资产收益率':'roe_yearly','年化总资产报酬率':'roa2_yearly','营业利润同比增长率':'op_yoy',\
          '销售净利润率同比增长':'netprofit_yoy', '营业收入同比增长率':'tr_yoy'      
         }
        '''
        cols = ['eps_yearly','inv_turn_yearly','ocfps_yearly','grossprofit_margin','expense_of_sales_yearly','roe_yearly','roa2_yearly',\
                'op_yoy','tr_yoy']
        fina = fina[cols]
        fina = fina.fillna(method='ffill') #向下填充  
        self.fina = fina.copy()
        return self.fina
    
        #存在缺失值或者负值（市盈率）情况下评分直接为0
        #营业收入增长率打分（0-10）
    def cal_tryoy(self,y):
        '''y是营业收入增长率'''
        try:
            return 5+ min(round(y-10),5) if y>=10 else 5+ max(round(y-10),-5)
        except:
            return 0

    #营业利润增长率打分(0-10)
    def cal_opyoy(self,y):
        '''y是营业利润增长率'''
        try:
            return 5+ min(round((y-20)/2),5) if y>=20 else 5+ max(round((y-20)/2),-5)
        except:
            return 0

    #毛利率打分
    def cal_gpm(self,y):
        '''y是最近季度毛利率-前三季度平均毛利率'''
        try:
            return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
        except:
            return 0
    #期间费用率打分
    def cal_exp(self,y):
        '''y是最近季度期间费用率-前三季度平均期间费用率'''
        try:
            return 5+min(round(y)/0.5,5) if y>0 else max(round(y)/0.5,-5)+5
        except:
            return 0
    #存货周转率打分
    def cal_inv(self,y):
        '''y是（最近季度存货周转率-前三季度平均存货周转率）/前三季度平均存货周转率*100'''
        try:
            return 5+min(round(y/2),5) if y>0 else max(round(y/2),-5)+5
        except:
            return 0
    #每股经营性现金流打分
    def cal_ocfp(self,y):
        '''y是（最近三季度每股经营性现金流之和-最近三季度每股收益之和）/最近三季度每股收益之和*100'''
        try:
            return 5+min(round(y/4),5) if y>0 else max(round(y/4),-5)+5
        except:
            return 0
    #净资产收益率打分
    def cal_roe(self,y):
        '''y是年化净资产收益率'''
        try:
            return 5+ min(round(y-15),5) if y>=15 else 5+ max(round(y-15),-5)
        except:
            return 0
    #总资产报酬率打分
    def cal_roa(self,y):
        '''y是最近季度年化总资产报酬率'''
        try:
            return min(round((y-5)/0.5),10) if y>=5 else max(round(y-5),0)
        except:
            return 0
    #计算财务指标得分
    def indicator_score(self,code):
        data = self.get_indicators(code)
        ''''(1)营业收入增长率打分'''
        data['营收得分'] = data['tr_yoy'].apply(self.cal_tryoy)
        '''(2)营业利润增长率打分'''
        data['利润得分'] = data['op_yoy'].apply(self.cal_opyoy)
        '''(3)毛利率打分'''
        #计算最近季度毛利率-前三季度平均毛利率
        data['gpm']=data['grossprofit_margin']-data['grossprofit_margin'].rolling(3).mean()
        data['毛利率得分']=data['gpm'].apply(self.cal_gpm)
        '''(4)期间费用率打分'''
        #最近季度期间费用率-前三季度平均期间费用率
        data['exp']=data['expense_of_sales_yearly']-data['expense_of_sales_yearly'].rolling(3).mean()
        data['费用得分']=data['exp'].apply(self.cal_exp)
        '''(5)周转率打分'''
        #（最近季度存货周转率-前三季度平均存货周转率）/前三季度平均存货周转率*100
        data['inv']=(data['inv_turn_yearly']-data['inv_turn_yearly'].rolling(3).mean())*100/data['inv_turn_yearly'].rolling(3).mean()
        data['周转得分']=data['inv'].apply(self.cal_inv)
        '''(6)每股经营现金流打分'''
        #（最近三季度每股经营性现金流之和-最近三季度每股收益之和）/最近三季度每股收益之和*100
        data['ocf']=(data['ocfps_yearly'].rolling(3).sum()-data['eps_yearly'].rolling(3).sum())*100/data['eps_yearly'].rolling(3).sum()
        data['现金得分']=data['ocf'].apply(self.cal_ocfp)
        '''(7)净资产收益率打分'''
        data['净资产得分']=data['roe_yearly'].apply(self.cal_roe)
        '''(8)总资产收益率打分'''
        data['总资产得分']=data['roa2_yearly'].apply(self.cal_roa)
        data = data.iloc[2:]
        #计算总得分
        data['总分']=data[['营收得分','利润得分','毛利率得分','费用得分','周转得分','现金得分','净资产得分','总资产得分']].sum(axis=1)
        return data[['营收得分','利润得分','毛利率得分','费用得分','周转得分','现金得分','净资产得分','总资产得分','总分']]
    # 计算所有股票财务指标总分
    def get_all_score(self):
        df=pd.DataFrame()
        for name,code in self.get_code().items():
            try:
                df[name]=indicator_score(code)['总分'].copy()
            except:
                continue
        return df.T.sort_values(dff.T.columns[-1],ascending=False).iloc[:10,-10:]# 获取最近日期总分排名前15个股


if __name__ == '__main__':
    s = Stock_Select('801017.SI',50,20)
    print(s.get_all_score().iloc[:,-1])
#  
        
        
        
    
    

牧原股份    41.0
民和股份    31.0
益生股份    29.0
圣农发展    21.0
罗牛山     13.0
仙坛股份    13.0
温氏股份    12.0
新五丰     10.0
Name: 20210930, dtype: float64


In [11]:
l = df['eps'] * [1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9,1,12/3,12/6,12/9]